In [ ]:
#Iterative Parsing
def count_tags(filename):
        d = {}
        for event, elem in ET.iterparse(filename):
            if not (elem.tag in d):
                d[elem.tag] = 1
            else:
                d[elem.tag] += 1
        print d
        return d

In [ ]:
#Tag Types
def key_type(element, keys):
    if element.tag == "tag":
        for e in element.iter("tag"):
            key = e.attrib['k']
            if not (lower.search(key) == None):
                keys['lower'] += 1
            elif not (lower_colon.search(key) == None):
                keys['lower_colon'] += 1
            elif not (problemchars.search(key) == None):
                keys['problemchars'] +=1
            else:
                keys['other'] += 1
    return keys

In [ ]:
#Exploring Users
def process_map(filename):
    users = set()
    for _, element in ET.iterparse(filename):
        if 'uid' in element.attrib:
            users.add(element.attrib['uid'])
    return users

In [ ]:
#Improving Street Names
# UPDATE THIS VARIABLE
mapping = { "St": "Street",
            "St.": "Street",
            "Ave": "Avenue",
            "Rd.": "Road"
            }


def audit_street_type(street_types, street_name):
    m = street_type_re.search(street_name)
    if m:
        street_type = m.group()
        if street_type not in expected:
            street_types[street_type].add(street_name)

In [ ]:
#Preparing For Database
def shape_element(element):
    node = {}
    
    node['created'] = {}
    node['pos'] = ["0","0"]
    if element.tag == "node" or element.tag == "way" :
        node['type'] = element.tag
        
        if element.tag == "way":
            node['node_refs'] = []
            for it in element.iter("nd"):
                node['node_refs'].append(it.attrib['ref'])

        for a in element.attrib:
            if a in ["version","changeset","timestamp","user","uid"]:
                node['created'][a] = element.attrib[a]
            elif a == "lat":
                node['pos'][0]=(float(element.attrib[a]))
            elif a == "lon":
                node['pos'][1]=(float(element.attrib[a]))
            else:
                node[a] = element.attrib[a]
        for it in element.iter("tag"):
            #Avoid problematic characters in the string
            e = problemchars.search(it.attrib['k'])
            if e:
                continue
            elif it.attrib['k'][:5] == "addr:":
                if 'address' not in node:
                    node['address'] = {}
                c = it.attrib['k'].split(':')
                if len(c) == 2:
                    node['address'][it.attrib['k'][5:]] = it.attrib['v']
            else:
                node[it.attrib['k']] = it.attrib['v']
        return node
    else:
        return None